In [7]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


# 데이터 읽어 들이기--- (※1)
#ci_year	ci_term	si_grad	si_ent_gubun	agescale	si_sex	si_last_hak	schoarYn	
#accept_point	avg_logincnt	avg_att_time	avg_att_rate	avg_bbscnt	avg_startlogin	entYn	classcnt	
#avg_te_avg	avg_te_passrate	gradcnt	counsel	stdResult
#features =14
#features =13
#filename="./dataset/20200612_2018_2019.csv"
#filename="./dataset/20200614_v1.csv"
#filename="./dataset/20200717_v1.csv"

features =20
filename="./dataset/20200605_v1.csv"

df_pre=pd.read_csv(filename,encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용
#df_pre=pd.read_csv('./dataset/20200612_app_point.csv',encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용





In [8]:
print("데이터의 총 수", len(df_pre))



데이터의 총 수 58088


In [9]:
df_pre.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58088 entries, 0 to 58087
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ci_year              58088 non-null  int64  
 1   ci_term              58088 non-null  int64  
 2   si_grad              58088 non-null  int64  
 3   si_ent_gubun         58088 non-null  int64  
 4   agescale             58088 non-null  int64  
 5   si_sex               58088 non-null  int64  
 6   si_last_hak          58088 non-null  int64  
 7   schoarYn             58088 non-null  int64  
 8   accept_point         58088 non-null  int64  
 9   avg_logincnt         58088 non-null  int64  
 10  avg_att_time         58088 non-null  int64  
 11  avg_att_rate         58088 non-null  float64
 12  avg_bbscnt           58088 non-null  int64  
 13  avg_startlogin       58088 non-null  int64  
 14  entYn                58088 non-null  int64  
 15  classcnt             58088 non-null 

In [10]:

df_pre.tail()

,ci_year,ci_term,si_grad,si_ent_gubun,agescale,si_sex,si_last_hak,schoarYn,accept_point,avg_logincnt,...,avg_att_rate,avg_bbscnt,avg_startlogin,entYn,classcnt,pre_avg_te_avg,pre_avg_te_passrate,pre_gradcnt,counsel,stdResult
58083,2019,2,4,2,50,1,3,1,70,74,...,100.00,0,30,0,6,3.42,100,1,0,0
58084,2019,2,4,2,50,2,3,1,70,50,...,99.72,0,20,0,8,3.67,100,1,0,0
58085,2019,2,4,2,40,2,3,1,70,63,...,100.00,6,22,0,6,3.33,100,1,0,0
58086,2019,2,4,2,40,2,2,1,70,98,...,100.00,12,47,0,8,3.67,100,1,0,0
58087,2019,2,4,2,30,1,2,1,70,80,...,100.00,0,33,0,7,3.58,100,1,0,0


In [11]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
df = df_pre.sample(frac=1)


dataset = df.values
X = dataset[:,0:features]
Y = dataset[:,features]

# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
#테스트, 검증데이터 분할 7:3 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

#데이터 정규화
X_train, X_test = X_train / 255, X_test / 255

from keras.layers import Dense, Dropout, Activation
# 모델 설정
model = Sequential()
model.add(Dense(28,  input_dim=features, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(loss='binary_crossentropy',
#model.compile(loss='mean_squared_error',
           optimizer='adam',
           # metrics=['accuracy'])
           metrics=['accuracy',recall_m,precision_m,f1_m])

# 학습 자동 중단 설정 성능이 개선되지 않는 epch 100번까지 수용
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)



# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건 설정
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)



# 모델 실행 및 저장
history = model.fit(X_train, Y_train, validation_split=0.20, epochs=500, batch_size=100, 
                    callbacks=[early_stopping_callback, checkpointer])

# 결과 출력
print("\n epoches-500, bat_size=100")
print("\n 학습중단 + 모델 성능개선 : early_stopping_callback:")
print("\n 예측정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))

Train on 32528 samples, validate on 8133 samples
Epoch 1/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.6247 - accuracy: 0.6680 - recall_m: 0.9153 - precision_m: 0.6513 - f1_m: 0.7570 - val_loss: 0.6085 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230

Epoch 00001: val_loss improved from inf to 0.60851, saving model to ./model/01-0.6085.hdf5
Epoch 2/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.5990 - accuracy: 0.7035 - recall_m: 0.9994 - precision_m: 0.7033 - f1_m: 0.8247 - val_loss: 0.5956 - val_accuracy: 0.7037 - val_recall_m: 0.9885 - val_precision_m: 0.7063 - val_f1_m: 0.8231

Epoch 00002: val_loss improved from 0.60851 to 0.59561, saving model to ./model/02-0.5956.hdf5
Epoch 3/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.5885 - accuracy: 0.7052 - recall_m: 0.9878 - precision_m: 0.7081 - f1_m: 0.8238 - val_loss: 0.5938 - val_accuracy: 0.7018 - val_recall_m:


Epoch 00023: val_loss did not improve from 0.44070
Epoch 24/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.4456 - accuracy: 0.7845 - recall_m: 0.8614 - precision_m: 0.8420 - f1_m: 0.8481 - val_loss: 0.4484 - val_accuracy: 0.7901 - val_recall_m: 0.7952 - val_precision_m: 0.8933 - val_f1_m: 0.8406

Epoch 00024: val_loss did not improve from 0.44070
Epoch 25/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.4479 - accuracy: 0.7811 - recall_m: 0.8539 - precision_m: 0.8445 - f1_m: 0.8448 - val_loss: 0.4523 - val_accuracy: 0.7564 - val_recall_m: 0.9215 - val_precision_m: 0.7742 - val_f1_m: 0.8406

Epoch 00025: val_loss did not improve from 0.44070
Epoch 26/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.4424 - accuracy: 0.7870 - recall_m: 0.8556 - precision_m: 0.8483 - f1_m: 0.8482 - val_loss: 0.4703 - val_accuracy: 0.7712 - val_recall_m: 0.7429 - val_precision_m: 0.9147 - val_f1_m: 0.8189

Epoch 00026: val_loss di

Epoch 48/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.4152 - accuracy: 0.8107 - recall_m: 0.8695 - precision_m: 0.8657 - f1_m: 0.8652 - val_loss: 0.4132 - val_accuracy: 0.8106 - val_recall_m: 0.8337 - val_precision_m: 0.8891 - val_f1_m: 0.8597

Epoch 00048: val_loss did not improve from 0.40808
Epoch 49/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.4134 - accuracy: 0.8140 - recall_m: 0.8736 - precision_m: 0.8655 - f1_m: 0.8676 - val_loss: 0.4142 - val_accuracy: 0.8119 - val_recall_m: 0.8377 - val_precision_m: 0.8873 - val_f1_m: 0.8609

Epoch 00049: val_loss did not improve from 0.40808
Epoch 50/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.4162 - accuracy: 0.8104 - recall_m: 0.8716 - precision_m: 0.8631 - f1_m: 0.8648 - val_loss: 0.4068 - val_accuracy: 0.8173 - val_recall_m: 0.8672 - val_precision_m: 0.8711 - val_f1_m: 0.8684

Epoch 00050: val_loss improved from 0.40808 to 0.40680, saving model to ./mo

32528/32528 [==============================] - 0s 11us/step - loss: 0.3915 - accuracy: 0.8273 - recall_m: 0.8963 - precision_m: 0.8639 - f1_m: 0.8786 - val_loss: 0.3894 - val_accuracy: 0.8327 - val_recall_m: 0.9182 - val_precision_m: 0.8532 - val_f1_m: 0.8839

Epoch 00072: val_loss did not improve from 0.38855
Epoch 73/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3947 - accuracy: 0.8261 - recall_m: 0.8948 - precision_m: 0.8638 - f1_m: 0.8775 - val_loss: 0.3877 - val_accuracy: 0.8335 - val_recall_m: 0.9208 - val_precision_m: 0.8527 - val_f1_m: 0.8848

Epoch 00073: val_loss improved from 0.38855 to 0.38771, saving model to ./model/73-0.3877.hdf5
Epoch 74/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3921 - accuracy: 0.8286 - recall_m: 0.8968 - precision_m: 0.8657 - f1_m: 0.8796 - val_loss: 0.3896 - val_accuracy: 0.8344 - val_recall_m: 0.9200 - val_precision_m: 0.8545 - val_f1_m: 0.8854

Epoch 00074: val_loss did not improve from 0.


Epoch 00096: val_loss improved from 0.38163 to 0.37970, saving model to ./model/96-0.3797.hdf5
Epoch 97/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3845 - accuracy: 0.8338 - recall_m: 0.9047 - precision_m: 0.8660 - f1_m: 0.8839 - val_loss: 0.3794 - val_accuracy: 0.8366 - val_recall_m: 0.8987 - val_precision_m: 0.8717 - val_f1_m: 0.8842

Epoch 00097: val_loss improved from 0.37970 to 0.37941, saving model to ./model/97-0.3794.hdf5
Epoch 98/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.3874 - accuracy: 0.8309 - recall_m: 0.9020 - precision_m: 0.8651 - f1_m: 0.8817 - val_loss: 0.3996 - val_accuracy: 0.8209 - val_recall_m: 0.8459 - val_precision_m: 0.8925 - val_f1_m: 0.8677

Epoch 00098: val_loss did not improve from 0.37941
Epoch 99/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.3885 - accuracy: 0.8293 - recall_m: 0.8991 - precision_m: 0.8651 - f1_m: 0.8803 - val_loss: 0.3880 - val_accuracy: 0.8252 - val

...


Epoch 00342: val_loss did not improve from 0.33527
Epoch 343/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3427 - accuracy: 0.8562 - recall_m: 0.9197 - precision_m: 0.8813 - f1_m: 0.8993 - val_loss: 0.3431 - val_accuracy: 0.8554 - val_recall_m: 0.9166 - val_precision_m: 0.8812 - val_f1_m: 0.8979

Epoch 00343: val_loss did not improve from 0.33527
Epoch 344/500
32528/32528 [==============================] - 1s 16us/step - loss: 0.3428 - accuracy: 0.8542 - recall_m: 0.9183 - precision_m: 0.8806 - f1_m: 0.8982 - val_loss: 0.3632 - val_accuracy: 0.8454 - val_recall_m: 0.8923 - val_precision_m: 0.8876 - val_f1_m: 0.8892

Epoch 00344: val_loss did not improve from 0.33527
Epoch 345/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.3431 - accuracy: 0.8548 - recall_m: 0.9182 - precision_m: 0.8807 - f1_m: 0.8982 - val_loss: 0.3605 - val_accuracy: 0.8451 - val_recall_m: 0.8765 - val_precision_m: 0.8995 - val_f1_m: 0.8871

Epoch 00345: val_loss

32528/32528 [==============================] - 0s 12us/step - loss: 0.3379 - accuracy: 0.8564 - recall_m: 0.9210 - precision_m: 0.8807 - f1_m: 0.8994 - val_loss: 0.3532 - val_accuracy: 0.8528 - val_recall_m: 0.9166 - val_precision_m: 0.8782 - val_f1_m: 0.8963

Epoch 00392: val_loss did not improve from 0.32937
Epoch 393/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3382 - accuracy: 0.8539 - recall_m: 0.9191 - precision_m: 0.8792 - f1_m: 0.8979 - val_loss: 0.3322 - val_accuracy: 0.8584 - val_recall_m: 0.9262 - val_precision_m: 0.8780 - val_f1_m: 0.9008

Epoch 00393: val_loss did not improve from 0.32937
Epoch 394/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3333 - accuracy: 0.8565 - recall_m: 0.9218 - precision_m: 0.8801 - f1_m: 0.8998 - val_loss: 0.3320 - val_accuracy: 0.8581 - val_recall_m: 0.9196 - val_precision_m: 0.8827 - val_f1_m: 0.9001

Epoch 00394: val_loss did not improve from 0.32937
Epoch 395/500
32528/32528 [=========

32528/32528 [==============================] - 0s 12us/step - loss: 0.3316 - accuracy: 0.8587 - recall_m: 0.9303 - precision_m: 0.8763 - f1_m: 0.9018 - val_loss: 0.3373 - val_accuracy: 0.8513 - val_recall_m: 0.8902 - val_precision_m: 0.8970 - val_f1_m: 0.8929

Epoch 00417: val_loss did not improve from 0.32680
Epoch 418/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.3335 - accuracy: 0.8577 - recall_m: 0.9293 - precision_m: 0.8764 - f1_m: 0.9012 - val_loss: 0.3322 - val_accuracy: 0.8613 - val_recall_m: 0.9457 - val_precision_m: 0.8674 - val_f1_m: 0.9043

Epoch 00418: val_loss did not improve from 0.32680
Epoch 419/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.3334 - accuracy: 0.8587 - recall_m: 0.9328 - precision_m: 0.8752 - f1_m: 0.9024 - val_loss: 0.3321 - val_accuracy: 0.8539 - val_recall_m: 0.9031 - val_precision_m: 0.8902 - val_f1_m: 0.8959

Epoch 00419: val_loss did not improve from 0.32680
Epoch 420/500
32528/32528 [=========

32528/32528 [==============================] - 0s 12us/step - loss: 0.3317 - accuracy: 0.8583 - recall_m: 0.9338 - precision_m: 0.8738 - f1_m: 0.9021 - val_loss: 0.3450 - val_accuracy: 0.8579 - val_recall_m: 0.9546 - val_precision_m: 0.8580 - val_f1_m: 0.9031

Epoch 00442: val_loss did not improve from 0.32496
Epoch 443/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3335 - accuracy: 0.8571 - recall_m: 0.9343 - precision_m: 0.8719 - f1_m: 0.9013 - val_loss: 0.3399 - val_accuracy: 0.8539 - val_recall_m: 0.9024 - val_precision_m: 0.8904 - val_f1_m: 0.8956

Epoch 00443: val_loss did not improve from 0.32496
Epoch 444/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.3332 - accuracy: 0.8586 - recall_m: 0.9330 - precision_m: 0.8747 - f1_m: 0.9022 - val_loss: 0.3297 - val_accuracy: 0.8554 - val_recall_m: 0.9261 - val_precision_m: 0.8746 - val_f1_m: 0.8989

Epoch 00444: val_loss did not improve from 0.32496
Epoch 445/500
32528/32528 [=========

32528/32528 [==============================] - 0s 11us/step - loss: 0.3323 - accuracy: 0.8582 - recall_m: 0.9323 - precision_m: 0.8750 - f1_m: 0.9020 - val_loss: 0.3277 - val_accuracy: 0.8636 - val_recall_m: 0.9503 - val_precision_m: 0.8670 - val_f1_m: 0.9062

Epoch 00492: val_loss did not improve from 0.32496
Epoch 493/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3284 - accuracy: 0.8596 - recall_m: 0.9348 - precision_m: 0.8745 - f1_m: 0.9030 - val_loss: 0.3283 - val_accuracy: 0.8550 - val_recall_m: 0.9337 - val_precision_m: 0.8687 - val_f1_m: 0.8994

Epoch 00493: val_loss did not improve from 0.32496
Epoch 494/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.3287 - accuracy: 0.8591 - recall_m: 0.9331 - precision_m: 0.8753 - f1_m: 0.9026 - val_loss: 0.3280 - val_accuracy: 0.8600 - val_recall_m: 0.9480 - val_precision_m: 0.8646 - val_f1_m: 0.9038

Epoch 00494: val_loss did not improve from 0.32496
Epoch 495/500
32528/32528 [=========

In [12]:
#테스트 데이터 검증
loss, accuracy, recall,precision,f1_socre  = model.evaluate(X_test, Y_test)
#accuracy  = model.evaluate(X_test, Y_test)

print("\n accuracy, precision, recall, f1_score")
print("\n  %.4f, %.4f, %4f, %.4f" % (accuracy, precision ,recall, f1_socre))

#print("\n accuracy")
#print("\n  %.4f % (accuracy)")

17427/17427 [==============================] - 0s 15us/step

 accuracy, precision, recall, f1_score

  0.8553, 0.8669, 0.936808, 0.8985
